In [0]:
# Import necessary libraries

from pyspark.sql.functions import col, lit, unix_timestamp, from_unixtime, to_timestamp

In [0]:
# Initialization Cell
s3_bucket = dbutils.widgets.get("s3_bucket")
s3_prefix = dbutils.widgets.get("s3_prefix")

# Confirm retrieval
print(f"Loaded s3_bucket: {s3_bucket}")
print(f"Loaded s3_prefix: {s3_prefix}")

Loaded s3_bucket: retail-intelligence-data
Loaded s3_prefix: raw/


In [0]:
# Define S3 path and read function
base_path = f"s3a://{s3_bucket}/{s3_prefix}"

def read_retail_csv(file_name):
    s3_path = f"{base_path}{file_name}"
    print(f"Reading: {s3_path}")
    return spark.read.format("csv").option("header","true").option("inferSchema","true").load(s3_path)

print(f"Base S3 path configured : {base_path}")
                                                                                    

Base S3 path configured : s3a://retail-intelligence-data/raw/


In [0]:
customers_df = read_retail_csv("olist_customers_dataset.csv").alias("cust")
orders_df = read_retail_csv("olist_orders_dataset.csv").alias("ord")
items_df = read_retail_csv("olist_order_items_dataset.csv").alias("item")
payments_df = read_retail_csv("olist_order_payments_dataset.csv").alias("pay")
reviews_df = read_retail_csv("olist_order_reviews_dataset.csv").alias("rev")
products_df = read_retail_csv("olist_products_dataset.csv").alias("prod")
sellers_df = read_retail_csv("olist_sellers_dataset.csv").alias("seller")
geo_df = read_retail_csv("olist_geolocation_dataset.csv").alias("geo")
category_df = read_retail_csv("product_category_name_translation.csv").alias("cat")

print("Retail tables loaded successfully")
print(f"Customers: {customers_df.count():,} rows")
print(f"Orders: {orders_df.count():,} rows")
print(f"Order Items: {items_df.count():,} rows")



Reading: s3a://retail-intelligence-data/raw/olist_customers_dataset.csv
Reading: s3a://retail-intelligence-data/raw/olist_orders_dataset.csv
Reading: s3a://retail-intelligence-data/raw/olist_order_items_dataset.csv
Reading: s3a://retail-intelligence-data/raw/olist_order_payments_dataset.csv
Reading: s3a://retail-intelligence-data/raw/olist_order_reviews_dataset.csv
Reading: s3a://retail-intelligence-data/raw/olist_products_dataset.csv
Reading: s3a://retail-intelligence-data/raw/olist_sellers_dataset.csv
Reading: s3a://retail-intelligence-data/raw/olist_geolocation_dataset.csv
Reading: s3a://retail-intelligence-data/raw/product_category_name_translation.csv
Retail tables loaded successfully
Customers: 99,441 rows
Orders: 99,441 rows
Order Items: 112,650 rows


In [0]:
# Perform joins to build the fact table

df_join = customers_df.join(orders_df, col("cust.customer_id")==col("ord.customer_id"), "inner").drop(orders_df["customer_id"])

df_join = df_join.join(items_df, col("ord.order_id")==col("item.order_id"),"inner").drop(items_df["order_id"])

df_join = df_join.join(products_df,col("item.product_id")==col("prod.product_id"),"left").drop(items_df["product_id"])

df_join = df_join.join(category_df,col("prod.product_category_name")==col("cat.product_category_name"),"left").drop(products_df["product_category_name"])

df_join = df_join.join(payments_df,col("ord.order_id")==col("pay.order_id"),"left").drop(payments_df["order_id"])

df_join = df_join.join(reviews_df.dropDuplicates(["order_id"]),col("ord.order_id")==col("rev.order_id"),"left")

df_join = df_join.join(sellers_df,col("item.seller_id")==col("seller.seller_id"),"left").drop(sellers_df["seller_id"])


#Geo-location was not joined to avoid inefficiency

print(f"Joined table row count is {df_join.count()}")


Joined table row count is 117604


In [0]:
df_join.columns

['customer_id',
 'customer_unique_id',
 'customer_zip_code_prefix',
 'customer_city',
 'customer_state',
 'order_id',
 'order_status',
 'order_purchase_timestamp',
 'order_approved_at',
 'order_delivered_carrier_date',
 'order_delivered_customer_date',
 'order_estimated_delivery_date',
 'order_item_id',
 'seller_id',
 'shipping_limit_date',
 'price',
 'freight_value',
 'product_id',
 'product_name_lenght',
 'product_description_lenght',
 'product_photos_qty',
 'product_weight_g',
 'product_length_cm',
 'product_height_cm',
 'product_width_cm',
 'product_category_name',
 'product_category_name_english',
 'payment_sequential',
 'payment_type',
 'payment_installments',
 'payment_value',
 'review_id',
 'order_id',
 'review_score',
 'review_comment_title',
 'review_comment_message',
 'review_creation_date',
 'review_answer_timestamp',
 'seller_zip_code_prefix',
 'seller_city',
 'seller_state']

In [0]:
# Drop redundant order_id column
df_join = df_join.drop(col("rev.order_id"))

In [0]:
# Drop product_category_name column as it is in portugese
df_join = df_join.drop("product_category_name")

In [0]:
df_join.columns

['customer_id',
 'customer_unique_id',
 'customer_zip_code_prefix',
 'customer_city',
 'customer_state',
 'order_id',
 'order_status',
 'order_purchase_timestamp',
 'order_approved_at',
 'order_delivered_carrier_date',
 'order_delivered_customer_date',
 'order_estimated_delivery_date',
 'order_item_id',
 'seller_id',
 'shipping_limit_date',
 'price',
 'freight_value',
 'product_id',
 'product_name_lenght',
 'product_description_lenght',
 'product_photos_qty',
 'product_weight_g',
 'product_length_cm',
 'product_height_cm',
 'product_width_cm',
 'product_category_name_english',
 'payment_sequential',
 'payment_type',
 'payment_installments',
 'payment_value',
 'review_id',
 'review_score',
 'review_comment_title',
 'review_comment_message',
 'review_creation_date',
 'review_answer_timestamp',
 'seller_zip_code_prefix',
 'seller_city',
 'seller_state']

In [0]:
display(df_join.limit(10))

customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,seller_id,shipping_limit_date,price,freight_value,product_id,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,seller_zip_code_prefix,seller_city,seller_state
9ef432eb6251297304e76186b10a928d,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,e481f51cbdc54678b7cc49136f2d6af7,delivered,10/2/2017 10:56,10/2/2017 11:07,10/4/2017 19:55,10/10/2017 21:25,10/18/2017 0:00,1,3504c0cb71d7fa48d967e0e4c94d59d9,10/6/2017 11:07,29.99,8.72,87285b34884572647811a353c7ac498a,40,268,4,500,19,8,13,housewares,2,voucher,1,18.59,a54f0611adc9ed256b57ede6b6eb5114,4,null,"Não testei o produto ainda, mas ele veio correto e em boas condições. Apenas a caixa que veio bem amassada e danificada, o que ficará chato, pois se trata de um presente.",10/11/2017 0:00,10/12/2017 3:43,9350,maua,SP
b0830fb4747a6c6d20dea0b8c802d7ef,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,53cdb2fc8bc7dce0b6741e2150273451,delivered,7/24/2018 20:41,7/26/2018 3:24,7/26/2018 14:31,8/7/2018 15:27,8/13/2018 0:00,1,289cdb325fb7e7f891c38608bf9e0962,7/30/2018 3:24,118.7,22.76,595fac2a385ac33a80bd5114aec74eb8,29,178,1,400,19,13,19,perfumery,1,boleto,1,141.46,8d5266042046a06655c8db133d120ba5,4,Muito boa a loja,Muito bom o produto.,8/8/2018 0:00,8/8/2018 18:37,31570,belo horizonte,SP
41ce2a54c0b03bf3443c3d931a367089,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,47770eb9100c2d0c44946d9cf07ec65d,delivered,8/8/2018 8:38,8/8/2018 8:55,8/8/2018 13:50,8/17/2018 18:06,9/4/2018 0:00,1,4869f7a5dfa277a7dca6462dcf3b52b2,8/13/2018 8:55,159.9,19.22,aa4383b373c6aca5d8797843e5594415,46,232,1,420,24,19,21,auto,1,credit_card,3,179.12,e73b67b67587f7644d5bd1a52deb1b01,5,null,null,8/18/2018 0:00,8/22/2018 19:07,14840,guariba,SP
f88197465ea7920adcdbec7375364d82,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,949d5b44dbf5de918fe9c16f97b45f8a,delivered,11/18/2017 19:28,11/18/2017 19:45,11/22/2017 13:39,12/2/2017 0:28,12/15/2017 0:00,1,66922902710d126a0e7d26b0e3805106,11/23/2017 19:45,45.0,27.2,d0b61bfb1de832b15ba9d266ca96e5b0,59,468,3,450,30,10,20,pet_shop,1,credit_card,1,72.2,359d03e676b3c069f62cadba8dd3f6e8,5,null,O produto foi exatamente o que eu esperava e estava descrito no site e chegou bem antes da data prevista.,12/3/2017 0:00,12/5/2017 19:21,31842,belo horizonte,MG
8ab97904e6daea8866dbdbc4fb7aad2c,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2/13/2018 21:18,2/13/2018 22:20,2/14/2018 19:46,2/16/2018 18:17,2/26/2018 0:00,1,2c9e548be18521d1c43cde1c582c6de8,2/19/2018 20:31,19.9,8.72,65266b2da20d04dbe00c5c2d3bb7859e,38,316,4,250,51,15,15,stationery,1,credit_card,1,28.62,e50934924e227544ba8246aeb3770dd4,5,null,null,2/17/2018 0:00,2/18/2018 13:02,8752,mogi das cruzes,SP
503740e9ca751ccdda7ba28e9ab8f608,80bb27c7c16e8f973207a5086ab329e2,86320,congonhinhas,PR,a4591c265e18cb1dcee52889e2d8acc3,delivered,7/9/2017 21:57,7/9/2017 22:10,7/11/2017 14:58,7/26/2017 10:57,8/1/2017 0:00,1,8581055ce74af1daba164fdbd55a40de,7/13/2017 22:10,147.9,27.36,060cb19345d90064d1015407193c233d,49,608,1,7150,65,10,65,auto,1,credit_card,6,175.26,89b738e70a1ce346db29a20fb2910161,4,null,null,7/27/2017 0:00,7/27/2017 22:48,7112,guarulhos,SP
ed0271e0b7da060a393796590e7b737a,36edbb3fb164b1f16485364b6fb04c73,98900,santa rosa,RS,136cce7faa42fdb2cefd53fdc79a6098,invoiced,4/11/2017 12:22,4/13/2017 13:25,null,null,5/9/2017 0:00,1,dc8798cbf453b7e0f98745e396cc5616,4/19/2017 13:25,49.9,16.05,a1804276d9941ac0733cfd409f5206eb,null,null,

In [0]:
# Final Transformation, Cleaning and Selection

final_df = df_join.select(
    # Customer
    col("cust.customer_unique_id").alias("customer_key"),
    col("cust.customer_city").alias("customer_city"),

    # Order
    # M = Month (1-12), d = Day (1-31), yyyy = Year, H = Hour (0-23), m = Minute
    col("ord.order_id").alias("order_id"),
    col("ord.order_status").alias("order_status"),
    to_timestamp(col("ord.order_purchase_timestamp"),"M/d/yyyy H:m").alias("purchase_date"),

    # Items
    col("item.seller_id").alias("seller_id"),
    col("item.price").alias("item_price"),
                 
    # Product
    col("cat.product_category_name_english").alias("product_category"),
    col("prod.product_weight_g").alias("product_weight_g"),
    col("prod.product_id").alias("product_id"),

    # Reviews
    col("rev.review_score").alias("review_score"),

    #Payment
    col("pay.payment_value").alias("total_payment_value"),
    col("pay.payment_type").alias("payment_method"),
    col("pay.payment_installments").alias("payment_installments")
)


print(f"Final dataframe created with {final_df.count():,} rows")

Final dataframe created with 117,604 rows


In [0]:
display(final_df.limit(10))

customer_key,customer_city,order_id,order_status,purchase_date,seller_id,item_price,product_category,product_weight_g,product_id,review_score,total_payment_value,payment_method,payment_installments
7c396fd4830fd04220f754e42b4e5bff,sao paulo,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02T10:56:00Z,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,housewares,500,87285b34884572647811a353c7ac498a,4,18.12,credit_card,1
af07308b275d755c9edb36a90c618231,barreiras,53cdb2fc8bc7dce0b6741e2150273451,delivered,2018-07-24T20:41:00Z,289cdb325fb7e7f891c38608bf9e0962,118.7,perfumery,400,595fac2a385ac33a80bd5114aec74eb8,4,141.46,boleto,1
3a653a41f6f9fc3d2a113cf8398680e8,vianopolis,47770eb9100c2d0c44946d9cf07ec65d,delivered,2018-08-08T08:38:00Z,4869f7a5dfa277a7dca6462dcf3b52b2,159.9,auto,420,aa4383b373c6aca5d8797843e5594415,5,179.12,credit_card,3
7c142cf63193a1473d2e66489a9ae977,sao goncalo do amarante,949d5b44dbf5de918fe9c16f97b45f8a,delivered,2017-11-18T19:28:00Z,66922902710d126a0e7d26b0e3805106,45.0,pet_shop,450,d0b61bfb1de832b15ba9d266ca96e5b0,5,72.2,credit_card,1
72632f0f9dd73dfee390c9b22eb56dd6,santo andre,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2018-02-13T21:18:00Z,2c9e548be18521d1c43cde1c582c6de8,19.9,stationery,250,65266b2da20d04dbe00c5c2d3bb7859e,5,28.62,credit_card,1
80bb27c7c16e8f973207a5086ab329e2,congonhinhas,a4591c265e18cb1dcee52889e2d8acc3,delivered,2017-07-09T21:57:00Z,8581055ce74af1daba164fdbd55a40de,147.9,auto,7150,060cb19345d90064d1015407193c233d,4,175.26,credit_card,6
36edbb3fb164b1f16485364b6fb04c73,santa rosa,136cce7faa42fdb2cefd53fdc79a6098,invoiced,2017-04-11T12:22:00Z,dc8798cbf453b7e0f98745e396cc5616,49.9,null,600,a1804276d9941ac0733cfd409f5206eb,2,65.95,credit_card,1
932afa1e708222e5821dac9cd5db4cae,nilopolis,6514b8ad8028c9f2cc2374ded245783f,delivered,2017-05-16T13:10:00Z,16090f2ca825584b5a147ab24aa30c86,59.99,auto,50,4520766ec412348b8d4caa5e8a18c464,5,75.16,credit_card,3
39382392765b6dc74812866ee5ee92a7,faxinalzinho,76c6e866289321a7c93b82b54852dc33,delivered,2017-01-23T18:29:00Z,63b9ae557efed31d1f7687917d248a8d,19.9,furniture_decor,300,ac1789e492dcd698c5c10b97a671243a,1,35.95,boleto,1
299905e3934e9e181bfb2e164dd4b4f8,sorocaba,e69bfb5eb88e0ed6a785585b27e16dbf,delivered,2017-07-29T11:55:00Z,7c67e1448b00f6e969d365cea6b010ab,149.99,office_furniture,9750,9a78fb9862b10749a117f7fc3c31f051,5,161.42,voucher,1


In [0]:
# Null values, Missing values and Duplicates

df = final_df.na.drop(subset=["order_id","customer_key","purchase_date","product_id"])
df = df.dropDuplicates()

print(f"Cleaned dataframe: {df.count():,} rows")
display(df.limit(10))


Cleaned dataframe: 106,388 rows


customer_key,customer_city,order_id,order_status,purchase_date,seller_id,item_price,product_category,product_weight_g,product_id,review_score,total_payment_value,payment_method,payment_installments
d975a54519948041a4365850cb594836,rio brilhante,d8b57220afe3f1416bb889cc0e61aa67,delivered,2018-03-12T17:23:00Z,8b321bb669392f5163d04c59e235e066,13.2,electronics,200,27fdfad2c6a8f91606eb41ea396785c6,3,31.43,credit_card,1
69438114b54f18813004b8a6249b19e7,belo horizonte,631e74e14da293fc4c6a8f02253aac6e,delivered,2018-08-26T11:23:00Z,cbd996ad3c1b7dc71fd0e5f5df9087e2,59.7,health_beauty,650,9749139e09929de55d37e68e64b89706,5,78.22,credit_card,4
731a7cfcd37a25cb4bbe606e4f0d2f87,belem,116debdd5705aaa51630c3e292565d06,delivered,2017-08-04T15:56:00Z,dbb9b48c841a0e39e21f98e1a6b2ec3e,24.99,housewares,350,b8762d562d810c2f8d7ad7e9c12957be,4,87.76,credit_card,8
6be6f25b5c6e48a53dbb51513ef097df,laguna,9df7c08a42d7b4f7a2d7c743a14948f8,delivered,2018-05-04T09:35:00Z,1464afc72f696af775557a821c2e253f,42.9,perfumery,135,7bcf2266b78ccc737b8f26f925d3c5d4,5,64.96,boleto,1
e3e79b00fec0242e78f6d0bc5a17a15f,osasco,d451da9b109e1786f303924d04ed72a1,delivered,2017-05-17T17:06:00Z,9646c3513289980f17226a2fc4720dbd,18.0,computers_accessories,150,f457c5f0c72ba4cae17636c5174add2e,5,26.27,boleto,1
aa28df52f6f4d313847d126b4db11717,belo horizonte,beca5b5e9460824d84352c01cb89b421,delivered,2018-01-30T11:03:00Z,4917cee8d902e13428c3ec4b1ca6f315,209.0,home_appliances_2,7500,0e687d00ba3214b4aca6397a14d49373,5,239.5,credit_card,2
c09049ee8be2608548fd850af7c0a532,suzano,c5f30bdd01bc931c09429dc3bba5ac58,delivered,2018-05-28T21:38:00Z,1c40343cc5d18c2d8248ac2f3366de34,116.5,perfumery,430,0e2d365a247d105fefc31b17ab8706d9,5,124.5,credit_card,1
9789c872c721af19e428c304beaf6472,brasilia,7a5472f7c8cecc2e1cf43d12271e4eca,delivered,2018-06-22T17:37:00Z,1a932caad4f9d804097d7f8e615baed1,118.0,construction_tools_construction,5450,779dd392d4fbe5ca656bf3ceabecbf0b,4,62.87,voucher,1
fbe8c53553220646741e6d717a55e114,sao paulo,88d482d263a239a1fa53ec9975895f8c,delivered,2018-03-04T16:01:00Z,609e1a9a6c2539919b8205cf7c4e6ff0,36.9,bed_bath_table,700,446a112adf10992c2e26eb566a68bd46,4,53.01,credit_card,1
1c1075866af31b51fb3c17ba99925102,jandira,7cf4d6735272dc5678263c8d9c1144d4,delivered,2017-09-27T21:30:00Z,3771c85bac139d2344864ede5d9341e3,52.8,null,800,2b7f2a7cf1025b94bc5dc6c28746beaf,4,62.74,credit_card,3


In [0]:
# Using Databricks Secrets for PostgreSQL connection
pg_host = dbutils.secrets.get(scope="postgres-secrets", key="host")
pg_port = dbutils.secrets.get(scope="postgres-secrets", key="port")
pg_database = dbutils.secrets.get(scope="postgres-secrets", key="database")
pg_username = dbutils.secrets.get(scope="postgres-secrets", key="username")
pg_password = dbutils.secrets.get(scope="postgres-secrets", key="password")

pg_table = "retail_fact_table" # Target table name

jdbc_url = f"jdbc:postgresql://{pg_host}:{pg_port}/{pg_database}"

connection_properties = {
    "user": pg_username,
    "password": pg_password,
    "driver": "org.postgresql.Driver"
}

print(f"Writing to PostgreSQL: {jdbc_url}")
print(f"Table: {pg_table}")
print(f"Rows: {df.count():,}")

# The final write command
try:
    df.write.mode("overwrite").jdbc(url=jdbc_url, table=pg_table, properties=connection_properties)
    print(f"ETL Complete! {final_df.count():,} records written to {pg_table}")
except Exception as e:
    print(f"Failed to write to PostgreSQL: {e}")
    raise

# Verification
verify_df = spark.read.jdbc(url=jdbc_url, table=pg_table, properties=connection_properties)
print(f"Read {verify_df.count():,} rows from {pg_table}")
display(verify_df.limit(10))

Writing to PostgreSQL: jdbc:[REDACTED]ql://[REDACTED]:[REDACTED]/[REDACTED]
Table: retail_fact_table
Rows: 106,388
ETL Complete! 117,604 records written to retail_fact_table
Read 106,388 rows from retail_fact_table


customer_key,customer_city,order_id,order_status,purchase_date,seller_id,item_price,product_category,product_weight_g,product_id,review_score,total_payment_value,payment_method,payment_installments
5179541696b879141c1f0bb532f988c9,curitiba,5447160f15621601fa9ba2b94f4906b3,delivered,2017-11-21T12:08:00Z,b33e7c55446eabf8fe1a42d037ac7d6d,146.0,watches_gifts,444,461f43be3bdf8844e65b62d9ac2c7a5a,5,161.77,credit_card,1
831a032a3327e2b8325faf9d37953870,americana,90349f264a3d6a2525a34598d09dda6b,delivered,2018-05-11T16:35:00Z,955fee9216a65b617aa5c0531780ce60,45.0,construction_tools_lights,200,216bb0e0cd43ffd832e0973d35e0377e,5,314.34,boleto,1
5143bcc6bc4a69be5dfd6db38f2c1bb3,belo horizonte,4630b9ea86d473a936d6dd589cee8b22,delivered,2018-06-27T16:11:00Z,e45edf75f568ed2f93f78d060e246e25,22.9,housewares,450,b61ca163d698bd09698b6876a3e945e6,5,38.16,credit_card,3
362f74c1d6e7015625b8464d05a8a149,sao paulo,c3716204ec5bc2bc5593bee858279148,delivered,2018-07-31T21:51:00Z,a3a38f4affed601eb87a97788c949667,69.9,sports_leisure,281,cd935d283d47f1050c505e1c39c48b67,1,88.49,credit_card,4
7d4c8f6ad946c29d7fe6500d6e7179ef,conselheiro mairinck,eea0eb694d4eb8ffa2ed9279f4c348e5,delivered,2018-03-08T16:00:00Z,4da0e408c99d2fdc2126dc9fce518060,67.0,sports_leisure,700,49c934af2be4194ac63262d60745fcb0,5,81.55,credit_card,1
5b586399e94f316df6602acbe6d80f70,sao paulo,cfb361b2049fa0ddeb6ca3959fa0767d,delivered,2018-08-20T23:06:00Z,3f995f07c49d0d55a99d5c54957f7d81,59.0,auto,450,b75a056d7d03227c3a3b60e9ccf518c9,5,67.57,credit_card,1
c6ffab32abf5dc36b10072de18d008af,sao jose dos campos,75679c6961aad55f81620d8542166135,delivered,2017-11-26T20:26:00Z,9f505651f4a6abe901a56cdc21508025,99.0,computers_accessories,400,163da86e86f0089de089399c28c3fcbb,5,107.89,credit_card,1
bb538da36f11aa0fd7c707f69fc85c3c,mogi das cruzes,26b66bcb4cb37fe553c114832d3a5e45,delivered,2018-04-11T18:17:00Z,59fb871bf6f4522a87ba567b42dafecf,199.99,health_beauty,400,5861173d9af66445f070078cfc29a597,5,209.15,credit_card,8
7baac76b8ecd2581992afa42a7544934,piranguinho,526043f9d39d510f7a79547410cd3318,delivered,2017-11-02T11:45:00Z,6560211a19b47992c3666cc44a7e94c0,49.0,watches_gifts,400,dc52f0f5d3ec37a93eaf956cde4e5d2c,4,64.1,credit_card,2
f7868a334063a6d3943900186a61dff4,guaruja,f8204e84e9a426029a3f9848738caf33,delivered,2018-08-05T15:33:00Z,d1c281d3ae149232351cd8c8cc885f0d,95.99,bed_bath_table,1150,ba18c48184ce58f0ffd8b656c4edf21c,4,118.48,credit_card,5
